In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchvision.models as models
import torch.utils.data as data
import sklearn.utils as utils
from PIL import Image
import torchvision.transforms as T
import sklearn.metrics as metrics

In [ ]:
input_path = '/kaggle/input/cassava-leaf-disease-classification/'

In [ ]:
df = pd.read_csv(os.path.join(input_path, 'train.csv'))

In [ ]:
df

In [ ]:
num_classes = len(df.label.unique())
num_classes

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.base = models.densenet121(pretrained=False)
        self.dense = nn.Linear(1000, num_classes)
    def forward(self, x):
        return self.dense(self.base(x))

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = Model().to(device)

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, df, transforms):
        super(Dataset, self).__init__()
        self.df = df
        self.transforms = transforms
    def __getitem__(self, index):
        row = self.df.iloc[index]
        label = row.label
        img = Image.open(f'{input_path}/train_images/{row.image_id}')
        return self.transforms(img), np.array([label])
    def __len__(self):
        return len(self.df)

In [ ]:
df = utils.shuffle(df, random_state=42)

In [ ]:
train_transforms = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5),)
])

In [ ]:
val_transforms = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5),)
])

In [ ]:
train_dataset = Dataset(df=df.iloc[:20000].reset_index(drop=True), transforms=train_transforms)
val_dataset = Dataset(df=df.iloc[20000:].reset_index(drop=True), transforms=val_transforms)

In [ ]:
train_dataloader = data.DataLoader(dataset=train_dataset, num_workers=2, shuffle=True, batch_size=32)
val_dataloader = data.DataLoader(dataset=val_dataset, num_workers=2, shuffle=True, batch_size=32)

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
opt = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
EPOCHS = 2

In [ ]:
for i in range(EPOCHS):
    model.train()
    print('\nTraining...')
    for b, (x, y) in enumerate(train_dataloader):
        
        opt.zero_grad()
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x)
        loss = criterion(y_pred, y.squeeze())
        loss.backward()
        opt.step()
        
        if b % 20 == 0:
            print('\rEpoch: {}/{}, Batch: {}/{} Loss: {}'.format(i+1, EPOCHS, b+1, len(train_dataloader), loss.item()), end='')
    model.eval()
    print('\nValidation....')
    y_actuals = []
    y_preds = []
    
    for b, (x, y) in enumerate(val_dataloader):
        
        x = x.cuda()
        y = y.cuda()
        y_pred = model(x).detach()
        loss = criterion(y_pred, y.squeeze())
        y_actuals += y.cpu().numpy().tolist()
        y_preds += y_pred.cpu().numpy().tolist()
        if b % 20 == 0:
            print('\rEpoch: {}/{}, Batch: {}/{} Loss: {}'.format(i+1, EPOCHS, b+1, len(val_dataloader), loss.item()), end='')
        
    y_actuals = np.array(y_actuals).squeeze()
    y_preds = np.array(y_preds).argmax(1)
    print('Accuracy: ', np.sum(y_preds == y_actuals) / len(y_actuals))
    print(metrics.classification_report(y_actuals, y_preds))

In [ ]:
submission_df = pd.read_csv(os.path.join(input_path, 'sample_submission.csv'))

In [ ]:
submission_df

In [ ]:
class TestDataset(data.Dataset):
    def __init__(self, df, transforms):
        super(TestDataset, self).__init__()
        self.df = df
        self.transforms = transforms
    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = Image.open(f'{input_path}/test_images/{row.image_id}')
        return self.transforms(img)
    def __len__(self):
        return len(self.df)

In [ ]:
test_transforms = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5),)
])

In [ ]:
train_dataset = TestDataset(df=submission_df, transforms=test_transforms)
test_dataloader = data.DataLoader(dataset=train_dataset, shuffle=False, batch_size=16)

In [ ]:
model.eval()
test_preds = []
for x in test_dataloader:
    x = x.cuda()
    y_pred = model(x).detach()
    test_preds += y_pred.cpu().numpy().argmax(1).tolist()
test_preds = np.array(test_preds)

In [ ]:
submission_df['label'] = test_preds

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!nvidia-smi